In [1]:
manualUpload = True #Define se irá usar upload do Drive (automático) ou da máquina (manual)

In [2]:
!pip install optuna --quiet
!pip install keras
!pip install ipywidgets
!pip install matplotlib
!pip install seaborn


[notice] A new release of pip available: 22.3.1 -> 23.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
!pip install tensorflow --user


[notice] A new release of pip available: 22.3.1 -> 23.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
from keras.layers import Input, Dropout, BatchNormalization, CuDNNGRU, CuDNNLSTM, LSTM, Dense, Bidirectional, Activation
from keras.models import Sequential, Model

from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
from IPython.display import display
from ipywidgets import IntProgress
from tensorflow import *

import matplotlib.pyplot as plt
import tensorflow as tf
#import keras

import seaborn as sns
import pandas as pd
import numpy as np
import optuna as op
import scipy as sp

import time
import io
import os

%matplotlib inline

In [5]:
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))

Device mapping: no known devices.


In [6]:
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

ModuleNotFoundError: No module named 'tensorflow'

In [18]:
import tensorflow.keras.backend as K
#from keras import backend as K
K.tensorflow_backend._get_available_gpus()

AttributeError: module 'tensorflow.keras.backend' has no attribute 'tensorflow_backend'

In [6]:
if manualUpload == False:
    from google.colab import drive, files
    drive.mount('/content/drive')

In [7]:
#Importação manual, caso necessária
# if not os.path.exists("TST.cod"):
#     if manualUpload == True:
#         upload = files.upload()
#         #decoding the files as uploaded will be a dictionary of keys (the file names) and values (the encoded file objects)
#         df_test = pd.read_table(io.StringIO(upload['TST.cod'].decode('utf-8')))

#     #Importação pelo Drive
#     else:
#         #decoding the files as uploaded will be a dictionary of keys (the file names) and values (the encoded file objects)
#         df_test = pd.read_table("/content/drive/MyDrive/Databases/TST.cod")
# else:
#     df_test = pd.read_table("TST.cod")

df_test = pd.read_table("TST.cod")

df_test.head()

,INDEX,UF_1,UF_2,UF_3,UF_4,UF_5,UF_6,UF_7,IDADE,SEXO_1,...,CEP4_7,CEP4_8,CEP4_9,CEP4_10,CEP4_11,CEP4_12,CEP4_13,CEP4_14,IND_BOM_1_1,IND_BOM_1_2
0,0,1,1,1,0,0,0,0,0.898745,1,...,0,1,1,1,0,0,0,0,1,0
1,1,1,1,0,1,0,0,0,0.847404,1,...,0,0,0,1,1,1,0,0,1,0
2,2,1,1,0,0,0,0,1,0.016513,0,...,0,1,0,0,1,0,0,1,0,1
3,3,1,1,0,1,0,0,0,0.688950,0,...,0,0,0,0,0,1,0,1,1,0
4,4,0,1,1,0,0,1,0,0.230114,1,...,0,1,0,0,1,0,1,0,1,0


In [8]:
df_test.head().T

,0,1,2,3,4
INDEX,0.0,1.0,2.0,3.0,4.0
UF_1,1.0,1.0,1.0,1.0,0.0
UF_2,1.0,1.0,1.0,1.0,1.0
UF_3,1.0,0.0,0.0,0.0,1.0
UF_4,0.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...
CEP4_12,0.0,1.0,0.0,1.0,0.0
CEP4_13,0.0,0.0,0.0,0.0,1.0
CEP4_14,0.0,0.0,1.0,1.0,0.0
IND_BOM_1_1,1.0,1.0,0.0,1.0,1.0


In [9]:
# #Importação manual, caso necessária
# if not os.path.exists("TRNcod.xls"):
#     if manualUpload == True:
#         upload = files.upload()
#         #decoding the files as uploaded will be a dictionary of keys (the file names) and values (the encoded file objects)
#         df_tr = pd.read_table(io.StringIO(upload['TRNcod.xls'].decode('utf-8')))

#     #Importação pelo Drive
#     else:
#         #decoding the files as uploaded will be a dictionary of keys (the file names) and values (the encoded file objects)
#         df_tr = pd.read_table("/content/drive/MyDrive/Databases/TRNcod.xls")
# else:
#     df_tr = pd.read_excel("TRNcod.xls")

df_tr = pd.read_table("TRNcod.xls") ##! <<<<<<<<<<<< READ_EXCEL NÃO FUNCIONA

# print(df_tr.columns.tolist())
df_tr.head()

,INDEX,UF_1,UF_2,UF_3,UF_4,UF_5,UF_6,UF_7,IDADE,SEXO_1,...,CEP4_7,CEP4_8,CEP4_9,CEP4_10,CEP4_11,CEP4_12,CEP4_13,CEP4_14,IND_BOM_1_1,IND_BOM_1_2
0,0,1,1,1,0,0,0,0,0.135098,1,...,0,0,1,1,0,1,1,1,0,1
1,1,1,0,1,0,0,1,0,0.273504,1,...,0,1,0,1,1,0,0,0,1,0
2,2,1,0,1,0,0,1,0,0.281910,0,...,1,1,0,0,0,0,1,0,1,0
3,3,1,1,1,0,0,0,0,0.225741,0,...,1,1,0,1,1,0,1,0,1,0
4,4,1,1,0,0,0,1,0,0.480403,0,...,1,1,1,0,0,1,0,1,1,0


In [10]:
'''
indarr = df_trein["IND_BOM_1_1"].to_numpy()
indarr2 = df_trein["IND_BOM_1_2"].to_numpy()

f = IntProgress(min = 0 , max = len(indarr)) # instantiate the bar
display(f) # display the bar

for i in range(len(indarr)):
    if indarr[i] == indarr2[i]:
        print(indarr[i], indarr2[i])
    if indarr[i] != 1 and indarr[i] != 0:
        print(f"indarr {i} = {indarr[i]}")
    if indarr2[i] != 1 and indarr2[i] != 0:
        print(f"indarr2 {i} = {indarr2[i]}")
    f.value += 1'''
#Utilizado para checar se IND_BOM_1_1 e IND_BOM_1_2 eram as duas classes separadas

'\nindarr = df_trein["IND_BOM_1_1"].to_numpy()\nindarr2 = df_trein["IND_BOM_1_2"].to_numpy()\n\nf = IntProgress(min = 0 , max = len(indarr)) # instantiate the bar\ndisplay(f) # display the bar\n\nfor i in range(len(indarr)):\n    if indarr[i] == indarr2[i]:\n        print(indarr[i], indarr2[i])\n    if indarr[i] != 1 and indarr[i] != 0:\n        print(f"indarr {i} = {indarr[i]}")\n    if indarr2[i] != 1 and indarr2[i] != 0:\n        print(f"indarr2 {i} = {indarr2[i]}")\n    f.value += 1'

In [11]:
if "IND_BOM_1_1" in df_tr:
    df_tr = df_tr.drop(columns = ["IND_BOM_1_1"])
if "INDEX" in df_tr:
    df_tr = df_tr.drop(columns = ["INDEX"])
if "IND_BOM_1_2" in df_tr:
    df_tr = df_tr.rename(columns = {"IND_BOM_1_2": "CLASSE_INDIV"})

df_tr.head()

#0 = Classe 1
#1 = Classe 2

,UF_1,UF_2,UF_3,UF_4,UF_5,UF_6,UF_7,IDADE,SEXO_1,NIVEL_RELACIONAMENTO_CREDITO01,...,CEP4_6,CEP4_7,CEP4_8,CEP4_9,CEP4_10,CEP4_11,CEP4_12,CEP4_13,CEP4_14,CLASSE_INDIV
0,1,1,1,0,0,0,0,0.135098,1,0.222222,...,0,0,0,1,1,0,1,1,1,1
1,1,0,1,0,0,1,0,0.273504,1,0.111111,...,0,0,1,0,1,1,0,0,0,0
2,1,0,1,0,0,1,0,0.281910,0,1.000000,...,0,1,1,0,0,0,0,1,0,0
3,1,1,1,0,0,0,0,0.225741,0,0.111111,...,0,1,1,0,1,1,0,1,0,0
4,1,1,0,0,0,1,0,0.480403,0,0.111111,...,1,1,1,1,0,0,1,0,1,0


In [13]:
df_tr_class_1 = df_tr[df_tr['CLASSE_INDIV'] == 0]
df_tr_class_2 = df_tr[df_tr['CLASSE_INDIV'] == 1]

df_tr_class_1_a = df_tr_class_1.sample(frac = 0.66)
df_tr_class_1_b = df_tr_class_1.drop(df_tr_class_1_a.index)

df_tr_class_2_a = df_tr_class_2.sample(frac = 0.66)
df_tr_class_2_b = df_tr_class_2.drop(df_tr_class_2_a.index)

df_tr_class_2_a = pd.concat([df_tr_class_2_a,
                     df_tr_class_2_a.sample(len(df_tr_class_1_a) - len(df_tr_class_2_a))],
                     ignore_index = True,
                     sort = False)

df_tr_class_2_b = pd.concat([df_tr_class_2_b,
                     df_tr_class_2_b.sample(len(df_tr_class_1_b) - len(df_tr_class_2_b))],
                     ignore_index = True,
                     sort = False)

df_trein = pd.concat([df_tr_class_1_a, df_tr_class_2_a])
df_valid = pd.concat([df_tr_class_1_b, df_tr_class_2_b])

print(len(df_tr_class_1_a), len(df_tr_class_2_b))

print("dados de treino e validação:")

df_tr_class_1 = None
df_tr_class_2 = None

df_tr_class_1_a = None
df_tr_class_1_b = None
df_tr_class_2_a = None
df_tr_class_2_b = None

df_trein = shuffle(df_trein).reset_index().drop(columns = ["index"])
df_valid = shuffle(df_valid).reset_index().drop(columns = ["index"])

print("dados de treino e validação:")
print(len(df_trein), len(df_valid))

168365 86733
dados de treino e validação:
dados de treino e validação:
336730 173466


In [14]:
df_treinX = df_trein.drop(columns = ["CLASSE_INDIV"])
df_treinY = df_trein["CLASSE_INDIV"]
df_validX = df_valid.drop(columns = ["CLASSE_INDIV"])
df_validY = df_valid["CLASSE_INDIV"]

In [15]:
patienceCallback = tf.keras.callbacks.EarlyStopping(monitor = 'loss', patience = 20)

#### Teste optuna

In [22]:
import optuna

from keras.backend import clear_session
from keras.datasets import mnist
from keras.layers import Conv2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop

In [29]:
N_TRAIN_EXAMPLES = 3000
N_VALID_EXAMPLES = 1000
BATCHSIZE = 2000
CLASSES = 10
EPOCHS = 100

In [60]:
def objective(trial):
    # Clear clutter from previous Keras session graphs.
    clear_session()
    
    #n_layers = trial.suggest_int("n_layers", 6, 9)
    #print(n_layers)
    
    dropout = trial.suggest_float("dropout", 0.21, 0.42)

    #hidden_layers = trial.suggest_categorical("hidden_layer_sizes", [(50,100),(100,100),(50,75,100),(25,50,75,100)])
    
    num_hidden = trial.suggest_int("n_units_l{}".format(4),  120, 250, log=True)
    print(num_hidden)
    
    MLPmodel = Sequential([
    Dense(num_hidden, activation = 'relu'),
    Dropout(rate=dropout),
    Dense(10, activation = trial.suggest_categorical("activation", ["relu", "sigmoid"])),
    ])
    
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-1, log=True)
    
    tf.keras.optimizers.Adam(
        learning_rate=learning_rate
    )
    tf.keras.optimizers.RMSprop(
        learning_rate=learning_rate
    )
    tf.keras.optimizers.SGD(
        learning_rate=learning_rate
    )
    
    loss_func = keras.losses.SparseCategoricalCrossentropy()
    
    MLPmodel.compile(optimizer = trial.suggest_categorical("optimizer", ["adam", "RMSprop", "SGD"]),
      loss = loss_func,
      metrics=['accuracy'],
      )
    
    MLPmodel.fit(df_treinX, df_treinY, epochs = EPOCHS, 
          batch_size = BATCHSIZE, callbacks = [patienceCallback],  
          validation_data = (df_validX, df_validY))

    

    """model.fit(
        x_train,
        y_train,
        validation_data=(x_valid, y_valid),
        shuffle=True,
        batch_size=BATCHSIZE,
        epochs=EPOCHS,
        verbose=False,
    )"""

    # Evaluate the model accuracy on the validation set.
    score = MLPmodel.evaluate(df_validX, df_validY, verbose=0)
    return score[1]

In [61]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10, timeout=600)

[I 2023-04-17 00:13:30,425] A new study created in memory with name: no-name-e527cfef-d1ba-4fd3-84ed-eaf0bc49b2e4


175
Epoch 1/100
169/169 [==============================] - 2s 9ms/step - loss: 0.8598 - accuracy: 0.5232 - val_loss: 0.6932 - val_accuracy: 0.5720
Epoch 2/100
169/169 [==============================] - 1s 7ms/step - loss: 0.7125 - accuracy: 0.5493 - val_loss: 0.6786 - val_accuracy: 0.5852
Epoch 3/100
169/169 [==============================] - 1s 6ms/step - loss: 0.6949 - accuracy: 0.5610 - val_loss: 0.6732 - val_accuracy: 0.5926
Epoch 4/100
169/169 [==============================] - 1s 6ms/step - loss: 0.6869 - accuracy: 0.5685 - val_loss: 0.6704 - val_accuracy: 0.5965
Epoch 5/100
169/169 [==============================] - 1s 6ms/step - loss: 0.6813 - accuracy: 0.5763 - val_loss: 0.6685 - val_accuracy: 0.5992
Epoch 6/100
169/169 [==============================] - 1s 6ms/step - loss: 0.6783 - accuracy: 0.5796 - val_loss: 0.6668 - val_accuracy: 0.6021
Epoch 7/100
169/169 [==============================] - 1s 6ms/step - loss: 0.6760 - accuracy: 0.5835 - val_loss: 0.6658 - val_accuracy: 0.

[I 2023-04-17 00:15:23,081] Trial 0 finished with value: 0.6274428367614746 and parameters: {'dropout': 0.37586147540947634, 'n_units_l4': 175, 'activation': 'sigmoid', 'learning_rate': 0.0006980401585250615, 'optimizer': 'SGD'}. Best is trial 0 with value: 0.6274428367614746.


155
Epoch 1/100
169/169 [==============================] - 2s 8ms/step - loss: 0.7454 - accuracy: 0.5374 - val_loss: 0.6926 - val_accuracy: 0.5700
Epoch 2/100
169/169 [==============================] - 1s 7ms/step - loss: 0.6989 - accuracy: 0.5597 - val_loss: 0.6867 - val_accuracy: 0.5862
Epoch 3/100
169/169 [==============================] - 1s 6ms/step - loss: 0.6826 - accuracy: 0.5963 - val_loss: 0.6574 - val_accuracy: 0.6141
Epoch 4/100
169/169 [==============================] - 1s 6ms/step - loss: 0.6569 - accuracy: 0.6177 - val_loss: 0.6574 - val_accuracy: 0.6252
Epoch 5/100
169/169 [==============================] - 1s 6ms/step - loss: 0.6484 - accuracy: 0.6271 - val_loss: 0.6425 - val_accuracy: 0.6299
Epoch 6/100
169/169 [==============================] - 1s 6ms/step - loss: 0.6435 - accuracy: 0.6325 - val_loss: 0.6450 - val_accuracy: 0.6285
Epoch 7/100
169/169 [==============================] - 1s 6ms/step - loss: 0.6402 - accuracy: 0.6346 - val_loss: 0.6433 - val_accuracy: 0.

[I 2023-04-17 00:16:31,323] Trial 1 finished with value: 0.6344125270843506 and parameters: {'dropout': 0.2519478753635454, 'n_units_l4': 155, 'activation': 'relu', 'learning_rate': 0.028896690436041778, 'optimizer': 'RMSprop'}. Best is trial 1 with value: 0.6344125270843506.


...

[I 2023-04-17 00:21:22,867] Trial 3 finished with value: 0.5 and parameters: {'dropout': 0.22075774817913546, 'n_units_l4': 223, 'activation': 'relu', 'learning_rate': 5.724244871775454e-05, 'optimizer': 'SGD'}. Best is trial 1 with value: 0.6344125270843506.


181
Epoch 1/100
169/169 [==============================] - 2s 12ms/step - loss: 0.9578 - accuracy: 0.4982 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 2/100
169/169 [==============================] - 2s 11ms/step - loss: 0.6933 - accuracy: 0.5014 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 3/100
169/169 [==============================] - 2s 10ms/step - loss: 0.6932 - accuracy: 0.5020 - val_loss: 0.6931 - val_accuracy: 0.5011
Epoch 4/100
169/169 [==============================] - 2s 10ms/step - loss: 0.6932 - accuracy: 0.5032 - val_loss: 0.6931 - val_accuracy: 0.5015
Epoch 5/100
169/169 [==============================] - 2s 10ms/step - loss: 0.6932 - accuracy: 0.5033 - val_loss: 0.6931 - val_accuracy: 0.5033
Epoch 6/100
169/169 [==============================] - 2s 10ms/step - loss: 0.6932 - accuracy: 0.5036 - val_loss: 0.6931 - val_accuracy: 0.5043
Epoch 7/100
169/169 [==============================] - 2s 10ms/step - loss: 0.6932 - accuracy: 0.5031 - val_loss: 0.6931 - val_accur

[I 2023-04-17 00:23:16,611] Trial 4 finished with value: 0.5052114129066467 and parameters: {'dropout': 0.39511715384342283, 'n_units_l4': 181, 'activation': 'relu', 'learning_rate': 0.0006214028639584695, 'optimizer': 'adam'}. Best is trial 1 with value: 0.6344125270843506.


219
Epoch 1/100
169/169 [==============================] - 3s 15ms/step - loss: 0.7640 - accuracy: 0.4839 - val_loss: 0.6932 - val_accuracy: 0.4784
Epoch 2/100
169/169 [==============================] - 2s 12ms/step - loss: 0.6953 - accuracy: 0.4871 - val_loss: 0.6932 - val_accuracy: 0.4803
Epoch 3/100
169/169 [==============================] - 2s 12ms/step - loss: 0.6943 - accuracy: 0.4878 - val_loss: 0.6932 - val_accuracy: 0.4807
Epoch 4/100
169/169 [==============================] - 2s 13ms/step - loss: 0.6938 - accuracy: 0.4887 - val_loss: 0.6931 - val_accuracy: 0.4810
Epoch 5/100
169/169 [==============================] - 2s 13ms/step - loss: 0.6937 - accuracy: 0.4878 - val_loss: 0.6931 - val_accuracy: 0.4819
Epoch 6/100
169/169 [==============================] - 2s 12ms/step - loss: 0.6936 - accuracy: 0.4921 - val_loss: 0.6931 - val_accuracy: 0.4953
Epoch 7/100
169/169 [==============================] - 2s 13ms/step - loss: 0.6935 - accuracy: 0.4929 - val_loss: 0.6931 - val_accur

[I 2023-04-17 00:26:38,846] Trial 5 finished with value: 0.4883492887020111 and parameters: {'dropout': 0.25404970246804226, 'n_units_l4': 219, 'activation': 'relu', 'learning_rate': 8.318054164825491e-05, 'optimizer': 'SGD'}. Best is trial 1 with value: 0.6344125270843506.


In [62]:
print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

Number of finished trials: 6
Best trial:
  Value: 0.6344125270843506
  Params: 
    dropout: 0.2519478753635454
    n_units_l4: 155
    activation: relu
    learning_rate: 0.028896690436041778
    optimizer: RMSprop


### Acurácia muito baixa, precisamos passar pelo processo de tunagem

In [21]:
MLPmodel = Sequential([
    Dense(246, activation = 'relu'),
    Dense(10, activation = 'sigmoid'),
    ])

tf.keras.optimizers.Adam(
    learning_rate=0.03
)

loss_func = keras.losses.SparseCategoricalCrossentropy()
loss2_func = tf.keras.losses.CategoricalCrossentropy()
MLPmodel.compile(optimizer = 'adam',
      loss = loss_func,
      metrics=['accuracy'],
      )

MLPmodel.fit(df_treinX, df_treinY, epochs = 10, 
          batch_size = 2000, callbacks = [patienceCallback],  
          validation_data = (df_validX, df_validY))

Epoch 1/10
169/169 [==============================] - 2s 8ms/step - loss: 0.6928 - accuracy: 0.6071 - val_loss: 0.6467 - val_accuracy: 0.6237
Epoch 2/10
169/169 [==============================] - 1s 5ms/step - loss: 0.6411 - accuracy: 0.6332 - val_loss: 0.6403 - val_accuracy: 0.6312
Epoch 3/10
169/169 [==============================] - 1s 5ms/step - loss: 0.6380 - accuracy: 0.6350 - val_loss: 0.6414 - val_accuracy: 0.6299
Epoch 4/10
169/169 [==============================] - 1s 5ms/step - loss: 0.6353 - accuracy: 0.6384 - val_loss: 0.6368 - val_accuracy: 0.6355
Epoch 5/10
169/169 [==============================] - 1s 5ms/step - loss: 0.6342 - accuracy: 0.6392 - val_loss: 0.6394 - val_accuracy: 0.6328
Epoch 6/10
169/169 [==============================] - 1s 5ms/step - loss: 0.6315 - accuracy: 0.6411 - val_loss: 0.6357 - val_accuracy: 0.6362
Epoch 7/10
169/169 [==============================] - 1s 5ms/step - loss: 0.6303 - accuracy: 0.6433 - val_loss: 0.6357 - val_accuracy: 0.6366
Epoch 

In [37]:
MLPmodel_2 = Sequential([
    Dense(128, activation = 'relu'),
    Dense(64, activation = 'sigmoid'),
    Dense(10, activation = 'sigmoid'),
    ])

MLPmodel_2.compile(optimizer = 'nadam',
      loss = 'mse',
      metrics=['accuracy'])

MLPmodel_2.fit(df_treinX, df_treinY, epochs = 10000, 
          batch_size = 2000, callbacks = [patienceCallback],  
          validation_data = (df_validX, df_validY))

Epoch 1/10000
169/169 [==============================] - 3s 8ms/step - loss: 0.2367 - accuracy: 0.1344 - val_loss: 0.2268 - val_accuracy: 0.1182
Epoch 2/10000
169/169 [==============================] - 1s 6ms/step - loss: 0.2255 - accuracy: 0.1216 - val_loss: 0.2241 - val_accuracy: 0.1543
Epoch 3/10000
169/169 [==============================] - 1s 6ms/step - loss: 0.2232 - accuracy: 0.1215 - val_loss: 0.2240 - val_accuracy: 0.1298
Epoch 4/10000
169/169 [==============================] - 1s 6ms/step - loss: 0.2217 - accuracy: 0.1219 - val_loss: 0.2219 - val_accuracy: 0.1366
Epoch 5/10000
169/169 [==============================] - 1s 6ms/step - loss: 0.2205 - accuracy: 0.1274 - val_loss: 0.2213 - val_accuracy: 0.1255
Epoch 6/10000
169/169 [==============================] - 1s 6ms/step - loss: 0.2194 - accuracy: 0.1256 - val_loss: 0.2213 - val_accuracy: 0.1390
Epoch 7/10000
169/169 [==============================] - 1s 6ms/step - loss: 0.2184 - accuracy: 0.1246 - val_loss: 0.2217 - val_ac

In [38]:
MLPmodel_2 = Sequential([
    Dense(64, activation = 'relu'),
    Dense(32, activation = 'sigmoid'),
    Dense(10, activation = 'sigmoid'),
    ])

MLPmodel_2.compile(optimizer = 'nadam',
      loss = 'mse',
      metrics=['accuracy'])

MLPmodel_2.fit(df_treinX, df_treinY, epochs = 10000, 
          batch_size = 1000, callbacks = [patienceCallback],  
          validation_data = (df_validX, df_validY))

Epoch 1/10000
337/337 [==============================] - 2s 4ms/step - loss: 0.2346 - accuracy: 0.1322 - val_loss: 0.2265 - val_accuracy: 0.1187
Epoch 2/10000
337/337 [==============================] - 1s 3ms/step - loss: 0.2251 - accuracy: 0.1090 - val_loss: 0.2238 - val_accuracy: 0.1082
Epoch 3/10000
337/337 [==============================] - 1s 3ms/step - loss: 0.2231 - accuracy: 0.1018 - val_loss: 0.2226 - val_accuracy: 0.1077
Epoch 4/10000
337/337 [==============================] - 1s 3ms/step - loss: 0.2219 - accuracy: 0.1043 - val_loss: 0.2218 - val_accuracy: 0.1063
Epoch 5/10000
337/337 [==============================] - 1s 3ms/step - loss: 0.2210 - accuracy: 0.1039 - val_loss: 0.2215 - val_accuracy: 0.1072
Epoch 6/10000
337/337 [==============================] - 1s 4ms/step - loss: 0.2203 - accuracy: 0.1069 - val_loss: 0.2213 - val_accuracy: 0.0979
Epoch 7/10000
337/337 [==============================] - 1s 3ms/step - loss: 0.2197 - accuracy: 0.1078 - val_loss: 0.2213 - val_ac

In [13]:
RFmodel = tfdf.keras.RandomForestModel()

RFmodel.fit(tfdf.keras.pd_dataframe_to_tf_dataset(df_trein, label = "CLASSE_INDIV"), 
          callbacks = [patienceCallback])

Use /tmp/tmppa5v88yw as temporary training directory
Reading training dataset...
Training dataset read in 0:01:24.607524. Found 336730 examples.
Training model...
Model trained in 0:19:47.842450
Compiling model...


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Model compiled.


In [14]:
RFmodel.summary()

Model: "random_forest_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 1
Trainable params: 0
Non-trainable params: 1
_________________________________________________________________
Type: "RANDOM_FOREST"
Task: CLASSIFICATION
Label: "__LABEL"

Input Features (243):
	ATIVIDADE_CONSUMIDOR_MERCADO_FINANCEIRO
	ATIVIDADE_EMAIL
	ATIVIDADE_ENDERECO
	ATIVIDADE_TELEFONE
	ATUALIZACAO_CONSUMIDOR_MERCADO_FINANCEIRO
	ATUALIZACAO_EMAIL
	ATUALIZACAO_ENDERECO
	BANCO_REST_IRPF_ULTIMA_1
	BANCO_REST_IRPF_ULTIMA_2
	BANCO_REST_IRPF_ULTIMA_3
	BANCO_REST_IRPF_ULTIMA_4
	BANCO_REST_IRPF_ULTIMA_5
	BANCO_REST_IRPF_ULTIMA_6
	BANCO_REST_IRPF_ULTIMA_7
	CAD_DEMOGRAFICO_VAR_0
	CAD_DEMOGRAFICO_VAR_1
	CAD_DEMOGRAFICO_VAR_10
	CAD_DEMOGRAFICO_VAR_11
	CAD_DEMOGRAFICO_VAR_13
	CAD_DEMOGRAFICO_VAR_14
	CAD_DEMOGRAFICO_VAR_15
	CAD_DEMOGRAFICO_VAR_16
	CAD_DEMOGRAFICO_VAR_17
	CAD_DEMOGRAFICO_VAR_19
	CAD_DEMOGRAFICO_VAR_2
	CAD_

In [17]:
RFmodel.compile(metrics = ["accuracy"])
RFmodel.evaluate(tfdf.keras.pd_dataframe_to_tf_dataset(df_valid, label = "CLASSE_INDIV"))

174/174 [==============================] - 55s 305ms/step - loss: 0.0000e+00 - accuracy: 0.6412


[0.0, 0.6412149667739868]

In [16]:
tfdf.model_plotter.plot_model_in_colab(RFmodel, tree_idx=0, max_depth = 3)